## Minor Project
### Crowd Classification using Deep Learning, Computer Vision and Decision Tree
### Guide: Dr. Sunil Kumar
### Students: Ananya Agrawal (199303010) & Hardik Srivastava (199303069)

## Step 1: Tensorflow inference to generate heatmap of crowd

Handling Imports

In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
import os
import random
import math
import sys
from pyheatmap.heatmap import HeatMap

2022-05-20 03:57:36.530766: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-20 03:57:36.538928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-20 03:57:36.538945: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Loading Model and Weights from disk

In [2]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = keras.models.model_from_json(loaded_model_json)

model.load_weights("weights.h5")

print("Loaded model from disk successfully")

Loaded model from disk successfully


2022-05-20 03:57:41.206210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-20 03:57:41.206316: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-20 03:57:41.206337: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bigfloppa): /proc/driver/nvidia/version does not exist
2022-05-20 03:57:41.206566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Setting up paths of image and other things

In [3]:
input_image = "/home/hardik/Projects/Minor-Project/new/IMG_233.jpg"

Loading and preprocessing input image to correspond to input layer size

In [4]:
img = np.array(cv2.imread(input_image, 0))
img = (img - 127.5) / 128
inputs = np.reshape(img, [1, 768, 1024, 1])

Inferencing through model

In [5]:
outputs = model.predict(inputs)
predicted_count = np.sum(outputs)

1/1 [==============================] - 0s 296ms/step


Printing predicted count

In [6]:
print("Predicted Count:", predicted_count)

Predicted Count: 27.358072


Generating density numpy array

In [7]:
y_p_den = np.reshape(outputs, (outputs.shape[1], outputs.shape[2]))

Generating Heatmap

In [8]:
den_resized = np.zeros((y_p_den.shape[0] * 4, y_p_den.shape[1] * 4))
for i in range(den_resized.shape[0]):
    for j in range(den_resized.shape[1]):
        den_resized[i][j] = y_p_den[int(i / 4)][int(j / 4)] / 16
den = den_resized
count = np.sum(den)
den = den * 10 / np.max(den)

crowd_img = cv2.imread(input_image, 1)

data = []
pts = []

for j in range(len(den)):
    for i in range(len(den[0])):
        for k in range(int(den[j][i])):
            data.append([i + 1, j + 1])
            pts.append((i + 1, j + 1))

hm = HeatMap(data, base=input_image)
hm.heatmap(save_as = 'output.png')
print("Heatmap Generated to output.png")

Heatmap Generated to output.png


### Step 2: Load data and create Decision Tree

Classification (Target Label Generation) using Decision Tree

Handling Imports

In [27]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

Loading Data from CSV

In [28]:
cluster_data = pd.read_csv("exported.csv", index_col=0)
cluster_data.head()

,IMG_NUM,SOURCE_DATASET,AB,AC,AD,AE,BC,BD,BE,CD,CE,DE,TARGET
0,28,A,226.214500,6.403124,174.287119,12.041595,86.815897,9.219544,6.324555,9.486833,7.071068,8.485281,dense
1,42,A,339.676317,62.801274,152.947703,70.710678,42.059482,57.454330,354.407957,84.504438,49.091751,385.149322,dense
2,341,A,71.168813,322.076078,307.483333,52.239832,234.326695,557.288076,306.778422,542.860940,486.864458,43.680659,dense
3,378,A,302.828334,21.633308,243.895469,37.483330,550.905618,728.715308,21.095023,294.183616,283.444880,493.852205,dense
4,14,A,26.172505,41.109610,287.141080,368.989160,29.732137,41.629317,31.622777,40.607881,215.520301,25.495098,dense


Preparing X and Y slices of data, where X is our source data and Y contains the Target Class

In [29]:
X = cluster_data.values[:, 2:-1]
Y = cluster_data.values[:, -1]

Splitting data into Train and Test

Setting up functions to create decision tree

In [34]:
# Function to perform training with giniIndex.
def train_using_gini(X_train, y_train):
    # Creating the classifier object
    clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, max_depth=None, min_samples_leaf=5)
    # Performing training
    clf_gini.fit(X_train, y_train)
    return clf_gini

# Function to make predictions
def prediction(X_test, clf_object):  
    # Predicton on test with giniIndex
    y_pred = clf_object.predict(X_test)
    print("Predicted values:")
    print(y_pred)
    return y_pred

In [35]:
clf_gini = train_using_gini(X, Y)


# y_pred_gini = prediction(X_test, clf_gini)
# cal_accuracy(y_test, y_pred_gini)

In [24]:
import numpy as np
import pandas as pd
from mat4py import loadmat
from sklearn.cluster import KMeans
import os
import math

In [25]:

# creates clusters of pts list
def make_clusters(pts):
    est = KMeans(5)
    est.fit(pts)
    y_kmeans = est.predict(pts)
    # cluster list
    cluster_list = [[], [], [], [], []]
    for index in range(len(pts)):
        cluster_list[y_kmeans[index]].append(pts[index])
    return cluster_list

# returns dist between 2 points
def pt_dist(p1, p2):
    xx = p1[0] - p2[0]
    yy = p1[1] - p2[1]
    return math.sqrt(xx*xx + yy*yy)

# returns min dist between cluster 1 and cluster 2
def min_dist(c1, c2):
    min = 9999999
    for p1 in c1:
        for p2 in c2:
            d = pt_dist(p1, p2)
            if min > d:
                min = d
    return min

all_clusters = make_clusters(pts)

ab = min_dist(all_clusters[0], all_clusters[1])
ac = min_dist(all_clusters[0], all_clusters[2])
ad = min_dist(all_clusters[0], all_clusters[3])
ae = min_dist(all_clusters[0], all_clusters[4])
bc = min_dist(all_clusters[1], all_clusters[2])
bd = min_dist(all_clusters[1], all_clusters[3])
be = min_dist(all_clusters[1], all_clusters[4])
cd = min_dist(all_clusters[2], all_clusters[3])
ce = min_dist(all_clusters[2], all_clusters[4])
de = min_dist(all_clusters[3], all_clusters[4])

In [26]:
check_entry = [ab, ac, ad, ae, bc, bd, be, cd, ce, ce]
print(check_entry)

[563.4696087634186, 81.00617260431454, 301.24076749337894, 1.0, 282.71894170713074, 1.0, 373.5906851087163, 1.0, 1.0, 1.0]


In [36]:
y_pred_gini = prediction([check_entry], clf_gini)

Predicted values:
['dense']
